In [2]:
import json
import os
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np
import re

In [3]:
from os import listdir
from os.path import isfile, join
dir_prefix = './data'
onlyfiles = [f for f in listdir(dir_prefix) if isfile(join(dir_prefix, f))]

In [4]:
neo_scraped = {}
for f in onlyfiles:
  neo = re.sub(r'_\d{4}', '',f)
  neo = neo.replace('.json', '')
  if neo == 'affluenza (1)':
    print(f)
  if neo not in neo_scraped:
    neo_scraped[neo] = os.stat(join(dir_prefix, f)).st_size
  else:
    neo_scraped[neo] += os.stat(join(dir_prefix, f)).st_size
neo_scraped = sorted(neo_scraped.items(), key=lambda kv: kv[1])
neo_scraped

[('manspread', 25886),
 ('mansplain', 395244),
 ('Rendezbooze', 528803),
 ('hundo', 625630),
 ('pwned', 1350910),
 ('knosh', 1763720),
 ('skrill', 1976074),
 ('fauxpology', 4257827),
 ('meep', 4310460),
 ('spime', 7030250),
 ('nom', 17815207),
 ('youthquake', 40565211),
 ('ninja_sex', 81236344),
 ('Rickroll', 220321421),
 ('party_foul', 224477624),
 ('humblebrag', 357459274),
 ('man_boobs', 372188635),
 ('jailbait', 395008876),
 ('fitspiration', 475787493),
 ('side_boob', 607069936),
 ('spliff', 1264216017),
 ('yeet', 1869224360),
 ('Nontroversy', 4913077730)]

In [5]:
YEAR_START = 2007
YEAR_END = 2019
tweet = {}
def read_yearly_tweets(word_list):
    trouble_shoot_empty = []
    entropy_df= pd.DataFrame()
    for word in word_list:
        print("reading %s ..." % word)
           
          ## if file is named with word + year
        for year in range(YEAR_START, YEAR_END+1):
            ##This code works, I commented it out because the local files on my machine done have years 2007 to 2019
            #data_file_name = "./data/" + word + "_" + str(year) + ".json"
            #if os.path.isfile(data_file_name):
                #if os.stat(data_file_name).st_size == 0:
                    #trouble_shoot_empty.append(word + "_" + str(year))
            #else:
                #with open(data_file_name, 'r') as f: 
                    #js = pd.read_json(f)
                    #entropy_df=pd.concat([entropy_df,js])
        #continue
      
      ## if file is named with just the word 
            data_file_name = "./data/" + word+ ".json"
            
            if os.path.isfile(data_file_name):
                if os.stat(data_file_name).st_size == 0:
                    trouble_shoot_empty.append(word + "_" + str(year))
                else:
            #js = pd.read_json(data_file_name)
                    whole_df = pd.read_json(data_file_name)
                    with open(data_file_name, 'r') as f: 
                        js1 = pd.read_json(f)
                        js1['neo_word']=word
                        entropy_df=pd.concat([entropy_df,js1])
                        
        
    return entropy_df

In [ ]:
smallest_k = 8
word_ls = [x[0] for x in neo_scraped[:smallest_k]] ## read the k files with the smallest memory
#word_ls.remove('affluenza (1)')
#word_ls += ['party_foul', 'affluenza', 'man_boobs', 'jailbait', 'genopolitics']# 'photobomb']
tweet = read_yearly_tweets(word_ls)

reading manspread ...
reading mansplain ...
reading Rendezbooze ...
reading hundo ...
reading pwned ...
reading knosh ...
reading skrill ...
reading fauxpology ...


## Mask neologism in tweet text ##

In [ ]:
def mask_neo_token(whole_tweet_df):
    whole_tweet_df['masked_text'] = whole_tweet_df.apply(lambda x: x.text.lower().replace(x.neo_word.lower(), '_NEOLOGISM_'), axis = 1)
    return whole_tweet_df

In [ ]:
entropy_df=mask_neo_token(tweet)

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
#Use spacy tokenizer to tokenize words in masked tweet. 
# Ouput: Spacy Doc
entropy_df['tokens'] = entropy_df['masked_text'].apply(lambda x: nlp.tokenizer(x))

In [ ]:
#Use regex to tokenize words in masked tweet
# Output: list of tokens without punctuation and whitespace
entropy_df['tokens_sans_punct'] = entropy_df['masked_text'].apply(lambda x: re.sub(r'[^\w\s]','',x).split(" "))

In [ ]:
#Count number of words in each tweet without punctuation and whitespace,  uses regex
entropy_df['count'] = entropy_df['masked_text'].apply(lambda x: len(re.sub(r'[^\w\s]','',x).split(" ")))

In [ ]:
entropy_df.head(100)

In [ ]:
Total_count=entropy_df['count'].sum()
Total_count

In [ ]:
#Make tokens into one big list of words from all tweets
from itertools import chain
words=list(chain(*entropy_df.tokens_sans_punct))

In [ ]:
tokenized_Sentences=[t.text for t in entropy_df['tokens']]

In [ ]:
from collections import Counter
def get_counts(tokens):
    counts=Counter()
    for token in tokens:
        counts[token]+=1
    return counts

In [ ]:
#Get count of all unique words from all tweets
token_counts=get_counts(words)

In [ ]:
#Remove '_NEOLOGISM_' and '_NEOLOGISM_ing' and white space from dictionary of word counts
token_counts.pop('_NEOLOGISM_', None)
token_counts.pop('_NEOLOGISM_ing', None)
token_counts.pop('', None)


In [ ]:
#Sort count dictionary 
import operator
sorted_x = sorted(token_counts.items(), key=operator.itemgetter(1), reverse=True)


In [ ]:
#Get dictionary of most K common words
K=500
most_common_500=dict()
for idx, (k, v) in enumerate(sorted_x[:K]):
    most_common_500[k]= v/Total_count
most_common_500

In [ ]:
#Group tweets by neologism, and concatenate all words used across tweets for each neologism
#Ouput: neologism and all words for that neologism
neologism_level_df=entropy_df.groupby(['neo_word'])['tokens_sans_punct'].apply(lambda x: list(chain(*x))).reset_index()

In [ ]:
neologism_level_df

In [ ]:
#Change words list for each neologism to include only unique words
#Removes repetition in entropy calculation. If a neologism has more words in it's tweets, it could be skewed if we don't obtain unique words
neologism_level_df['tokens_sans_punct']=neologism_level_df['tokens_sans_punct'].apply(lambda x: pd.unique(x).tolist())

In [ ]:
#Remove "_NEOLOGISM_" and"_NEOLOGISM_ing" from list of words for each neologism. 
neologism_level_df['tokens_sans_punct'] = neologism_level_df.tokens_sans_punct.apply(lambda lst: [x for x in lst if x != "_NEOLOGISM_"])
neologism_level_df['tokens_sans_punct'] = neologism_level_df.tokens_sans_punct.apply(lambda lst: [x for x in lst if x != "_NEOLOGISM_ing"])


In [ ]:
neologism_level_df

In [ ]:
import numpy as np

In [ ]:
def make_neologism_level_list(df):
    entropy_list=[]
    for i,v in enumerate(df.tokens_sans_punct):
        entropy=0
        for word in v:
            if word in most_common_500.keys():
                entropy+=(-most_common_500[word])*np.log2(most_common_500[word])
            else:
                entropy+=0
        entropy_list.append(entropy)
    return entropy_list

In [ ]:
neologism_level_df['entropy']=make_neologism_level_list(neologism_level_df)

In [ ]:
neologism_level_df